In [140]:
#!pip install sympy

In [141]:
from pyomo.environ import *
import numpy as np
from math import pi

In [142]:
Sbase = 10 # MW

buses = {
    1: [1, 0, 1.00, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [2, 1, 1.01, 0.0, 0.0, 0.0, 0.0, 0.0],
    3: [3, 2, 1.00, 0.0, 0.0, 0.0, 0.3, 0.1]
}

lines = {
    1: [1, 2, 0.0192, 0.0575, 0.0264, 1, 30/Sbase],
    2: [1, 3, 0.0452, 0.1852, 0.0204, 1, 30/Sbase],
    3: [2, 3, 0.0570, 0.1737, 0.0184, 1, 30/Sbase]
}

gens = {
    1: [1, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.00375, 2, 0],
    2: [2, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.0175, 1.75, 0]
}

In [143]:
nb = len(buses)
nl = len(lines)
ng = len(gens)

Ybus = np.zeros((nb, nb), dtype=np.complex128)
g = np.zeros((nb, nb))
b = np.zeros((nb,nb))
line_collection = []
# Loop through lines
for lineid, linedata in lines.items():

  i = linedata[0]-1
  k = linedata[1]-1
  line_collection.append((i,k))

  Z = linedata[2] + 1j*linedata[3]
  Bs = 1j*linedata[4]
  a = linedata[5]

  Ybus[i][i] += (1/(Z*a**2))
  Ybus[k][k] += (1/(Z*a**2))

  Ybus[i][i] += Bs
  Ybus[k][k] += Bs

  Ybus[i][k] -= 1/(a*Z)
  Ybus[k][i] -= 1/(a*Z)

  b[i][k] = Bs.imag
  b[k][i] = Bs.imag

G = Ybus.real
B = Ybus.imag

print(Ybus)
print(G)
print(B)

[[ 6.46838347-20.69594776j -5.22464618+15.64672684j
  -1.24373729 +5.09602092j]
 [-5.22464618+15.64672684j  6.9301765 -20.79930607j
  -1.70553032 +5.19737923j]
 [-1.24373729 +5.09602092j -1.70553032 +5.19737923j
   2.9492676 -10.25460015j]]
[[ 6.46838347 -5.22464618 -1.24373729]
 [-5.22464618  6.9301765  -1.70553032]
 [-1.24373729 -1.70553032  2.9492676 ]]
[[-20.69594776  15.64672684   5.09602092]
 [ 15.64672684 -20.79930607   5.19737923]
 [  5.09602092   5.19737923 -10.25460015]]


In [144]:
print(P_G)
print(P_G_bound)
print(list(gens.keys()))

(P_G0, P_G1)
[[0.0, 2.0], [0.0, 2.0]]
[1, 2]


In [145]:
def var_list_insert(var_list, bound_list, variable_list, Var_bound_list):
    for i,item in enumerate(var_list):
        variable_list.append(item)
        Var_bound_list.append(bound_list[i])
    return variable_list, Var_bound_list

In [ ]:
import sympy as sp

# ---------------------------
# 1. 基本尺寸与索引
# ---------------------------
n_buses = nb          # 母线个数
n_lines = nl          # 线路个数
buses_range = range(n_buses)
#line_collection         # e.g. [(0,1), (1,2), ...]，长度为 n_lines

ref_bus = 0            # 参考母线 index，例如 0

# 电压幅值上下限、热稳定上限（只用于 box bound，不进 AL）
V_sq_min = [0.9 for i in buses_range]
V_sq_max = [1.1 for i in buses_range]
S_tot_sq_max = [lines[line][6] ** 2 for line in lines]


# ---------------------------
# 2. 决策变量（SymPy symbols）
#    这里为了简单：对每个母线都定义 P_G, Q_G
#    若某母线不是发电机母线，可以把相应成本系数设为 0
# ---------------------------
variable_list = []
Var_bound_list = []

P_G = sp.symbols(f'P_G0:{len(gens)}')
P_G_bound = [[gens[gen][1], gens[gen][2]] for gen in gens]
variable_list, Var_bound_list = var_list_insert(P_G, P_G_bound, variable_list, Var_bound_list)

Q_G = sp.symbols(f'Q_G0:{len(gens)}')
Q_G_bound = [[gens[gen][3], gens[gen][4]]  for gen in gens] 
variable_list, Var_bound_list = var_list_insert(Q_G, Q_G_bound, variable_list, Var_bound_list)

V_R = sp.symbols(f'V_R0:{n_buses}')
V_R_bound = [[0.9, 1.1] for bus in range(nb)]
variable_list, Var_bound_list = var_list_insert(V_R, V_R_bound, variable_list, Var_bound_list)

V_I = sp.symbols(f'V_I0:{n_buses}')
V_I_bound = [[0.9, 1.1] for bus in range(nb)]
variable_list, Var_bound_list = var_list_insert(V_I, V_I_bound, variable_list, Var_bound_list)


V_sq = sp.symbols(f'V_sq0:{n_buses}')     # 用来表示 𝔙_i = |V_i|^2
V_sq_bound = [[0.81, 1.21] for bus in range(nb)]
variable_list, Var_bound_list = var_list_insert(V_sq, V_sq_bound, variable_list, Var_bound_list)

P_ij = sp.symbols(f'P_ij0:{n_lines}')
P_ij_bound = [[-lines[line][6], lines[line][6]] for line in lines]
variable_list, Var_bound_list = var_list_insert(P_ij, P_ij_bound, variable_list, Var_bound_list)

Q_ij = sp.symbols(f'Q_ij0:{n_lines}')
Q_ij_bound = [[-lines[line][6], lines[line][6]] for line in lines]
variable_list, Var_bound_list = var_list_insert(Q_ij, Q_ij_bound, variable_list, Var_bound_list)

S_tot_sq = sp.symbols(f'S_tot_sq0:{n_lines}')  # S^{total,sq}_{ij}
S_tsq_bound = [[0, lines[line][6]**2] for line in lines]
variable_list, Var_bound_list = var_list_insert(S_tot_sq, S_tsq_bound, variable_list, Var_bound_list)

# ---------------------------
# 3. 参数（可以是 float / SymPy symbol / Matrix）
#    这里只给出结构，具体数据你在外面填
# ---------------------------

# 发电机成本系数：C_i(P_Gi) = 0.5*a_i P_Gi^2 + b_i P_Gi + c_i
a_cost = [1 for i in buses_range]
b_cost = [2 for i in buses_range]
c_cost = [1 for i in buses_range]

# 负荷
P_D = [buses[bus][6] for bus in buses] # data [sp.symbols(f'P_D{i}') for i in buses]
Q_D = [buses[bus][7] for bus in buses] # data [sp.symbols(f'Q_D{i}') for i in buses]

# 母线导纳矩阵 Y = G + jB
G_mat = G
B_mat = B

# 支路参数：对每条线 ℓ = 0..n_lines-1
g_line = g
b_line = b


# ---------------------------
# 4. 拉格朗日乘子 λ 和罚参数 ρ
#    每类等式约束对应一组 λ 和一个 ρ
# ---------------------------

# (1) 有功/无功功率平衡 λ
lambda_P_bal = [0 for n in range(n_buses)] #initial sp.symbols(f'lam_P_bal0:{n_buses}')
lambda_Q_bal = [0 for n in range(n_buses)] #initial sp.symbols(f'lam_Q_bal0:{n_buses}')
rho_P_bal = 10 #initial #sp.symbols('rho_P_bal')
rho_Q_bal = 10 #initial sp.symbols('rho_Q_bal')

# (2) 线流定义约束
lambda_P_flow = [0 for i in range(n_lines)] #initial sp.symbols(f'lam_P_flow0:{n_lines}')
lambda_Q_flow = [0 for i in range(n_lines)] #initial sp.symbols(f'lam_Q_flow0:{n_lines}')
rho_P_flow = 1 #initial sp.symbols('rho_P_flow')
rho_Q_flow = 1 #initial sp.symbols('rho_Q_flow')

# (3) 电压平方定义 𝔙_i = (V_i^R)^2 + (V_i^I)^2
lambda_Vsq = [0 for i in range(n_buses)] #initial sp.symbols(f'lam_V_sq0:{n_buses}')
rho_Vsq = 0.1 #initial sp.symbols('rho_V_sq')

# (4) 视在功率平方定义 S_tot_sq = P_ij^2 + Q_ij^2
lambda_Ssq = [0 for i in range(n_lines)] #initial sp.symbols(f'lam_S_sq0:{n_lines}')
rho_Ssq = 1 #initial sp.symbols('rho_S_sq')

# (5) 参考母线虚部 V_I_ref = 0
lambda_ref = 0 #initial sp.symbols('lam_ref')
rho_ref = 100 #initial sp.symbols('rho_ref')

# ---------------------------
# 5. 目标函数：发电成本 sum_i C_i(P_Gi)
# ---------------------------
obj = 0
for i in range(len(gens.keys())):
    PGi = P_G[i]
    obj += 0.5 * a_cost[i] * PGi**2 + b_cost[i] * PGi + c_cost[i]

# ---------------------------
# 6. 构建各类等式约束 h(x) = 0
#    然后写入 Augmented Lagrangian
# ---------------------------

L_aug = obj  # 从原始目标开始

# ---- (1) 有功功率平衡约束 ----
for i in buses_range:
    ViR = V_R[i]
    ViI = V_I[i]

    # sum_j ( G_ij VjR - B_ij VjI )
    sum_GR_BI = 0
    sum_GI_BR = 0
    for j in buses_range:
        VjR = V_R[j]
        VjI = V_I[j]
        Gij = G_mat[i][j]
        Bij = B_mat[i][j]
        sum_GR_BI += Gij * VjR - Bij * VjI
        sum_GI_BR += Gij * VjI + Bij * VjR

    P_inj = ViR * sum_GR_BI + ViI * sum_GI_BR


    if i in list(gens.keys()):
        h_P_bal_i = P_G[i-1] - P_D[i] - P_inj  # 对应式 (2)
    else:
        h_P_bal_i = P_D[i] - P_inj  # 对应式 (2)

    L_aug += lambda_P_bal[i] * h_P_bal_i + (rho_P_bal / 2) * h_P_bal_i**2

# ---- (2) 无功功率平衡约束 ----
for i in buses_range:
    ViR = V_R[i]
    ViI = V_I[i]

    sum_GR_BI = 0
    sum_GI_BR = 0
    for j in buses_range:
        VjR = V_R[j]
        VjI = V_I[j]
        Gij = G_mat[i][j]
        Bij = B_mat[i][j]
        sum_GR_BI += Gij * VjR - Bij * VjI
        sum_GI_BR += Gij * VjI + Bij * VjR

    Q_inj = ViI * sum_GR_BI - ViR * sum_GI_BR

    if i in list(gens.keys()):
        h_Q_bal_i = Q_G[i-1] - Q_D[i] - Q_inj  # 对应式 (3)
    else:
        h_Q_bal_i = - Q_D[i] - Q_inj  # 对应式 (3)

    L_aug += lambda_Q_bal[i] * h_Q_bal_i + (rho_Q_bal / 2) * h_Q_bal_i**2

# ---- (3) 支路潮流定义约束 P_ij, Q_ij ----
for ell, (i, j) in enumerate(line_collection):
    ViR = V_R[i]
    ViI = V_I[i]
    VjR = V_R[j]
    VjI = V_I[j]
    g_ij = g_line[i][j]
    b_ij = b_line[i][j]

    # P_ij 定义式 (4)
    P_expr = (
        ViR * (g_ij * (ViR - VjR) - b_ij * (ViI - VjI))
        + ViI * (g_ij * (ViI - VjI) + b_ij * (ViR - VjR))
    )

    # Q_ij 定义式 (5)
    Q_expr = (
        ViI * (g_ij * (ViR - VjR) - b_ij * (ViI - VjI))
        - ViR * (g_ij * (ViI - VjI) + b_ij * (ViR - VjR))
    )

    h_P_flow_ell = P_ij[ell] - P_expr
    h_Q_flow_ell = Q_ij[ell] - Q_expr

    L_aug += lambda_P_flow[ell] * h_P_flow_ell + (rho_P_flow / 2) * h_P_flow_ell**2
    L_aug += lambda_Q_flow[ell] * h_Q_flow_ell + (rho_Q_flow / 2) * h_Q_flow_ell**2

# ---- (4) 电压平方定义 𝔙_i = (V_i^R)^2 + (V_i^I)^2 ----
for i in buses_range:
    h_Vsq_i = V_sq[i] - (V_R[i]**2 + V_I[i]**2)  # 对应式 (6)
    L_aug += lambda_Vsq[i] * h_Vsq_i + (rho_Vsq / 2) * h_Vsq_i**2

# ---- (5) 支路视在功率平方定义 S_tot_sq = P_ij^2 + Q_ij^2 ----
for ell in range(n_lines):
    h_Ssq_ell = S_tot_sq[ell] - (P_ij[ell]**2 + Q_ij[ell]**2)  # 对应式 (7)
    L_aug += lambda_Ssq[ell] * h_Ssq_ell + (rho_Ssq / 2) * h_Ssq_ell**2

# ---- (6) 参考母线约束 V_ref^I = 0 ----
h_ref = V_I[ref_bus]  # = 0  对应式 (8)
L_aug += lambda_ref * h_ref + (rho_ref / 2) * h_ref**2

# 至此，L_aug 就是 ACOPF 的 augmented Lagrangian（仅含 box 约束）


IndexError: tuple index out of range

In [ ]:
L_aug

0.5*P_G0**2 + 2*P_G0 + 0.5*P_G1**2 + 2*P_G1 + 0.5*P_G2**2 + 2*P_G2 + 50.0*V_I0**2 + 5.0*(P_G0 - V_I0*(6.46838346734966*V_I0 - 5.22464617988566*V_I1 - 1.24373728746401*V_I2 - 20.6959477620055*V_R0 + 15.6467268408034*V_R1 + 5.09602092120209*V_R2) - V_R0*(20.6959477620055*V_I0 - 15.6467268408034*V_I1 - 5.09602092120209*V_I2 + 6.46838346734966*V_R0 - 5.22464617988566*V_R1 - 1.24373728746401*V_R2))**2 + 5.0*(P_G1 - V_I1*(-5.22464617988566*V_I0 + 6.93017649658468*V_I1 - 1.70553031669903*V_I2 + 15.6467268408034*V_R0 - 20.7993060690599*V_R1 + 5.19737922825651*V_R2) - V_R1*(-15.6467268408034*V_I0 + 20.7993060690599*V_I1 - 5.19737922825651*V_I2 - 5.22464617988566*V_R0 + 6.93017649658468*V_R1 - 1.70553031669903*V_R2))**2 + 0.5*(P_ij0 - V_I0*(0.0264*V_R0 - 0.0264*V_R1) - V_R0*(-0.0264*V_I0 + 0.0264*V_I1))**2 + 0.5*(-P_ij0**2 - Q_ij0**2 + S_tot_sq0)**2 + 0.5*(P_ij1 - V_I0*(0.0204*V_R0 - 0.0204*V_R2) - V_R0*(-0.0204*V_I0 + 0.0204*V_I2))**2 + 0.5*(-P_ij1**2 - Q_ij1**2 + S_tot_sq1)**2 + 0.5*(P_ij2 - V

In [ ]:
from qhdopt import QHD

model = QHD.SymPy(L_aug, [x, y, z])

NameError: name 'f' is not defined